# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Store csv created in part one into a DataFrame
Weather_data_df = pd.read_csv("../Part I - WeatherPy/Output Data/Cities.csv")
Weather_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,astorga,-23.2325,-51.6656,79.03,31.0,91.0,14.43,BR,1.657900e+09
1,1,victor harbor,-35.5667,138.6167,54.00,56.0,100.0,16.17,AU,1.657900e+09
2,2,nikolskoye,59.7035,30.7861,65.98,58.0,100.0,5.75,RU,1.657900e+09
3,3,ushuaia,-54.8000,-68.3000,35.26,55.0,20.0,8.05,AR,1.657900e+09
4,4,barrow,71.2906,-156.7887,33.82,86.0,100.0,11.50,US,1.657900e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [28]:
# Store locations and Humidity
locations = Weather_data_df[["Lat", "Lng"]].astype(float)
humidity = Weather_data_df["Humidity"].astype(float)

# Create humidity heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, 
                                weights=humidity,
                                dissipating=False,
                                max_intensity=200, 
                                point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# Filter for Max temp lower than 27 degrees and greater than 21 degrees.
filtered_weather_df = Weather_data_df.loc[(Weather_data_df['Max Temp'] >= 70) & (Weather_data_df['Max Temp'] <=80)]

# Wind speed less than 15 kph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 15]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,7,bethel,41.3712,-73.4140,79.90,49.0,0.0,1.01,US,1.657900e+09
12,13,albany,42.6001,-73.9662,76.91,45.0,0.0,1.99,US,1.657900e+09
40,46,luanda,-8.8368,13.2343,75.20,69.0,0.0,10.36,AO,1.657900e+09
54,61,urumqi,43.8010,87.6005,77.95,26.0,0.0,2.24,CN,1.657900e+09
88,98,westport,41.1415,-73.3579,79.81,52.0,0.0,9.22,US,1.657900e+09
90,101,thompson,55.7435,-97.8558,71.76,56.0,0.0,3.44,CA,1.657900e+09
95,106,jamestown,42.0970,-79.2353,74.12,52.0,0.0,4.61,US,1.657900e+09
104,115,centerville,39.6284,-84.1594,79.66,59.0,0.0,5.75,US,1.657900e+09
169,183,moosomin,50.1333,-101.6677,73.26,73.0,0.0,4.68,CA,1.657900e+09
232,250,morondava,-20.2833,44.2833,72.36,63.0,0.0,11.21,MG,1.657900e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
# Create hotel_df and Hotel Name column
hotel_df = pd.DataFrame(filtered_weather_df, columns= ["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,bethel,US,41.3712,-73.4140,
12,albany,US,42.6001,-73.9662,
40,luanda,AO,-8.8368,13.2343,
54,urumqi,CN,43.8010,87.6005,
88,westport,US,41.1415,-73.3579,
90,thompson,CA,55.7435,-97.8558,
95,jamestown,US,42.0970,-79.2353,
104,centerville,US,39.6284,-84.1594,
169,moosomin,CA,50.1333,-101.6677,
232,morondava,MG,-20.2833,44.2833,


In [59]:
hotel_df = filtered_weather_df

# Create params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": gkey
}

# Loop through data
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from data frame
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Create base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel for {row['City']} is: {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No result... skipping.")
        
    print("------------")

Closest hotel for bethel is: Danbury.
------------
Closest hotel for albany is: Voorheesville.
------------
Closest hotel for luanda is: Luanda.
------------
Closest hotel for urumqi is: Ürümqi.
------------
Closest hotel for westport is: Norwalk.
------------
Closest hotel for thompson is: Thompson.
------------
Closest hotel for jamestown is: Jamestown.
------------
Closest hotel for centerville is: Centerville.
------------
Closest hotel for moosomin is: Moosomin.
------------
Closest hotel for morondava is: Morondava.
------------
Closest hotel for tukrah is: Tocra.
------------
Closest hotel for san ramon de la nueva oran is: Orán.
------------
Closest hotel for coswig is: Dresden.
------------
Closest hotel for boissevain is: Boissevain.
------------
Closest hotel for koungou is: Mamoudzou.
------------
Closest hotel for menongue is: Menongue.
------------
Closest hotel for edmundston is: Edmundston.
------------


In [60]:
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,7,bethel,41.3712,-73.4140,79.90,49.0,0.0,1.01,US,1.657900e+09,Danbury
12,13,albany,42.6001,-73.9662,76.91,45.0,0.0,1.99,US,1.657900e+09,Voorheesville
40,46,luanda,-8.8368,13.2343,75.20,69.0,0.0,10.36,AO,1.657900e+09,Luanda
54,61,urumqi,43.8010,87.6005,77.95,26.0,0.0,2.24,CN,1.657900e+09,Ürümqi
88,98,westport,41.1415,-73.3579,79.81,52.0,0.0,9.22,US,1.657900e+09,Norwalk


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [62]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))